# Dimension Tables

Creates and maintains dimension tables for the homelab monitoring data warehouse:
* `dim_node` - Proxmox nodes
* `dim_date_hour` - Date and hour combinations for time-based analysis

# Populate Dimension Tables

In [0]:
-- POPULATE DIM_NODE from existing data
-- Merge nodes from both node_status and lxc_status silver tables
MERGE INTO homelab.gold.dim_node AS target
USING (
  SELECT DISTINCT
    node_name,
    MIN(partition_date) as first_seen_date,
    MAX(partition_date) as last_seen_date
  FROM (
    SELECT node_name, partition_date FROM homelab.silver.proxmox_node_status
    UNION ALL
    SELECT node_name, partition_date FROM homelab.silver.proxmox_lxc_status
  )
  GROUP BY node_name
) AS source
ON target.node_name = source.node_name
WHEN MATCHED THEN UPDATE SET
  last_seen_date = source.last_seen_date
WHEN NOT MATCHED THEN INSERT (
  node_name,
  first_seen_date,
  last_seen_date,
  is_current
) VALUES (
  source.node_name,
  source.first_seen_date,
  source.last_seen_date,
  true
);

In [0]:
-- POPULATE DIM_DATE_HOUR with a generated date range
-- Generates all date/hour combinations for past 1 year (365 days)
-- Run this periodically to extend the range as needed
MERGE INTO homelab.gold.dim_date_hour AS target
USING (
  SELECT DISTINCT
    date_val as report_date,
    hour_val as report_hour,
    CONCAT(CAST(date_val AS STRING), ' ', LPAD(CAST(hour_val AS STRING), 2, '0'), ':00') as date_hour_string,
    date_format(date_val, 'EEEE') as day_of_week,
    dayofweek(date_val) as day_of_week_num,
    weekofyear(date_val) as week_of_year,
    month(date_val) as month_num,
    date_format(date_val, 'MMMM') as month_name,
    quarter(date_val) as quarter,
    year(date_val) as year,
    CASE WHEN dayofweek(date_val) IN (1, 7) THEN true ELSE false END as is_weekend,
    hour_val as hour_of_day
  FROM (
    SELECT 
      date_add(current_date(), -365 + seq) as date_val,
      hour_seq as hour_val
    FROM 
      (SELECT explode(sequence(0, 365)) as seq) dates
      CROSS JOIN
      (SELECT explode(sequence(0, 23)) as hour_seq) hours
  )
) AS source
ON target.report_date = source.report_date AND target.report_hour = source.report_hour
WHEN NOT MATCHED THEN INSERT (
  report_date,
  report_hour,
  date_hour_string,
  day_of_week,
  day_of_week_num,
  week_of_year,
  month_num,
  month_name,
  quarter,
  year,
  is_weekend,
  hour_of_day
) VALUES (
  source.report_date,
  source.report_hour,
  source.date_hour_string,
  source.day_of_week,
  source.day_of_week_num,
  source.week_of_year,
  source.month_num,
  source.month_name,
  source.quarter,
  source.year,
  source.is_weekend,
  source.hour_of_day
);